In [2]:
!pip install yfinance
!pip install yahooquery
!pip install tvdatafeed
!pip install tensorflow
!pip install --upgrade mplfinance
from tvDatafeed import TvDatafeed, Interval
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
import mplfinance as mpl 
from datetime import datetime
import glob
from PIL import Image
import cv2
import gc
import gc
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('agg')
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Activation,Dropout,Flatten,Dense,AveragePooling2D,GlobalAveragePooling2D
!pip install -U git+https://github.com/twopirllc/pandas-ta
import pandas_ta as pta


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/twopirllc/pandas-ta to /tmp/pip-req-build-7pisgfdj
  Running command git clone -q https://github.com/twopirllc/pandas-ta /tmp/pip-req-build-7pisgfdj


In [1]:

def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")
  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")
  if not os.path.exists("/content/checkpoints/"):
    os.mkdir("/content/checkpoints/")


def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols


def download_data(symbols, periodd, intervall):
  indexx = 100
  work_with_dir()
  for symbol in symbols:
    if ((symbols.index(symbol)+1) % 100 == 0):
      print(f" -- {indexx}", end="")
      indexx = indexx + 100
    try:
        data = yf.download(symbol, period=periodd,
                           interval=intervall, progress=False, show_errors=False)
        if data.empty:
           pass
        else:
            data.to_csv(f"/content/data/{symbol}.csv")
    except:
       print("Error!")
  print(" ")


def extract_data(how_many_future_candles, how_many_past_candles, each_row_past):
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
  pd.options.mode.chained_assignment = None
  files = os.listdir("/content/data/")
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  index = 1
  for file in files:
     print(f"File Number {index}:", end=" ")
     each_file_proc(file, now, how_many_future_candles,
                    how_many_past_candles, each_row_past)
     index = index + 1
  print(" ")
  return now


def each_file_proc(file, now, how_many_future_candles, how_many_past_candles, each_row_past):

    address = f"/content/data/{file}"
    data = pd.read_csv(address)
    data = data.dropna()
    dfs = []
    suggs = []
    if len(data.columns) == 7:
      data = data.iloc[:, 1:]
    data["6_rsi"] =  pta.rsi(data['Close'], length = 6)
    data["14_rsi"] = pta.rsi(data['Close'], length = 14)
    data["26_rsi"] = pta.rsi(data['Close'], length = 26)
    data["52_rsi"] = pta.rsi(data['Close'], length = 52)

    data.drop(columns=['Adj Close', 'Volume'], inplace=True)
    data = np.array(data)
    data = data.astype(float)
    for i in range(each_row_past, data.shape[0]-which_future_or_past):

        rows = data[i-each_row_past:i, :]
        past_candles = []
        for z in range(1, how_many_past_candles+1):
          past_candles.append((data[i-z][3]+data[i-z][0])/2)
        past_candles = sum(past_candles)/len(past_candles)
        next_candles = []
        for z in range(0, how_many_future_candles):
          next_candles.append((data[i+z][3]+data[i+z][0])/2)
        next_candles = sum(next_candles)/len(next_candles)
        if next_candles > past_candles:
          suggs.append(1)
        else:
          suggs.append(0)

        x = pd.DataFrame(rows, columns=["Open", "High", "Low", "Close","6_rsi", "14_rsi", "26_rsi","52_rsi"])
        dfs.append(x)

    index = 0
    all_n = np.array(dfs).shape[0]
    for df in dfs:
        date_list = []
        rrng = np.array(df).shape[0]
        for i in range(rrng):
            date_list.append(datetime.fromordinal(
                datetime.today().toordinal()+i).strftime('%Y-%m-%d'))
        df.index = date_list
        df.index.name = "Date"
        df.index = pd.to_datetime(df.index)
        right_now = datetime.now().strftime("%H%M%S%f")
        address = f"/content/extracted/{now}/{right_now}_{suggs[index]}.png"
        
        #apds = [ 
        # mpl.make_addplot(df["6_rsi"],color='b',width=2),
        # mpl.make_addplot(df["14_rsi"],color='g',width=2),
        # mpl.make_addplot(df["26_rsi"],color='r',width=2),
        # mpl.make_addplot(df["52_rsi"],color='y',width=2),
       #]

        fig, _a = mpl.plot(df, type="candle", style="yahoo", axisoff=True,
                            returnfig=True, tight_layout=True,figsize =(1.5,1.5))
        fig.savefig(address)
        plt.close("all")

        index = index + 1
        if index % 10 == 0:
            print(f"{index}/{all_n}", end=" ")
        if index % 160 == 0:
          print(" ")

    print("")
    del data, dfs, suggs, df, date_list
    gc.collect()


def start(how_many_future_candles, how_many_past_candles, each_row_past):
    folder_name = extract_data(
        how_many_future_candles, how_many_past_candles, each_row_past)
    return folder_name


In [3]:
symbols = get_crypto_syms()
print(f"Symbols : {len(symbols)}")
symbols = ["btc-usd","eth-usd","trx-usd","ltc-usd","xrp-usd","bnb-usd"]
download_data(symbols,"max","1wk")

Symbols : 1500
Data Folder Removed
 


In [9]:
how_many_future_candles = 3
how_many_past_candles = 1
each_row_past = 150

global which_future_or_past
which_future_or_past = None
if how_many_future_candles > how_many_past_candles:
    which_future_or_past = how_many_future_candles
else:
    which_future_or_past = how_many_past_candles

folder_name = start(how_many_future_candles,how_many_past_candles,each_row_past)
len(os.listdir(f"/content/extracted/{folder_name}"))

Files In Data : 6
Processing File:
File Number 1: 10/258 20/258 30/258 40/258 50/258 60/258 70/258 80/258 90/258 100/258 110/258 120/258 130/258 140/258 150/258 160/258  
170/258 180/258 190/258 200/258 210/258 220/258 230/258 240/258 250/258 
File Number 2: 10/94 20/94 30/94 40/94 50/94 60/94 70/94 80/94 90/94 
File Number 3: 10/258 20/258 30/258 40/258 50/258 60/258 70/258 80/258 90/258 100/258 110/258 120/258 130/258 140/258 150/258 160/258  
170/258 180/258 190/258 200/258 210/258 220/258 230/258 240/258 250/258 
File Number 4: 10/94 20/94 30/94 40/94 50/94 60/94 70/94 80/94 90/94 
File Number 5: 10/94 20/94 30/94 40/94 50/94 60/94 70/94 80/94 90/94 
File Number 6: 10/94 20/94 30/94 40/94 50/94 60/94 70/94 80/94 90/94 
 


892

In [10]:
shutil.make_archive(folder_name,"zip",f"/content/extracted/{folder_name}/")
#shutil.unpack_archive("/content/extracted/084505.zip","/content/extracted/455")

'/content/124132.zip'

In [11]:
#folder_name = '112904'

label = []
data  = []
files = os.listdir(f"/content/extracted/{folder_name}/")
for i, image_name in enumerate(files):
  if image_name.split(".")[1] == "png":
    image = cv2.imread(f"/content/extracted/{folder_name}"+"/"+image_name)

    dim = (120, 90)
    resized = cv2.resize(image, dim)
    data.append(np.array(resized))
    sugg = image_name.split("_")[1].split(".")[0]
    label.append(int(sugg))
data = np.array(data)
data = data / 255
print(data.shape)
xTrain , xTest , yTrain , yTest = train_test_split(data,label,test_size=0.2,random_state=99)

data = None
label = None
print(f"xTrain : {len(xTrain)} \\ xTest : {len(xTest)}")

nytrain = []
nytest = []
yn = 0
nn = 0

for i in yTrain:
  if i == 1:
    nytrain.append([1,0])
    yn += 1
  else:
    nytrain.append([0,1])
    nn += 1
for i in yTest:
  if i == 1:
    nytest.append([1,0])
    yn += 1
  else:
    nytest.append([0,1])
    nn += 1

yTrain = np.array(nytrain)
yTest = np.array(nytest)

print(f"yn: {yn} nn: {nn}")

(892, 90, 120, 3)
xTrain : 713 \ xTest : 179
yn: 461 nn: 431


In [12]:
model = Sequential()

model.add(Conv2D(128,    (3, 3),activation="relu", input_shape=(xTrain.shape[1], xTrain.shape[2],3)))
model.add(Conv2D(100,     (2, 2),activation="relu",))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64,    (3, 3),activation="relu",))
model.add(Conv2D(32,    (2, 2),activation="relu",))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64,    (3, 3),activation="relu",))
model.add(Conv2D(32,    (2, 2),activation="relu",))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(750,activation="relu"))
model.add(Dense(1024,activation="relu"))
model.add(Dense(1024,activation="relu"))
model.add(Dense(1024,activation="relu"))
model.add(Dense(2,activation="sigmoid"))

adamax = tf.keras.optimizers.Adamax(
    learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=adamax,
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 88, 118, 128)      3584      
                                                                 
 conv2d_7 (Conv2D)           (None, 87, 117, 100)      51300     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 43, 58, 100)      0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 41, 56, 64)        57664     
                                                                 
 conv2d_9 (Conv2D)           (None, 40, 55, 32)        8224      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 20, 27, 32)       0         
 2D)                                                  

In [13]:
filepath = "/content/checkpoints/{epoch:02d}-{val_accuracy:.2f}.h5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

#model.fit(xTrain,yTrain,batch_size=64,epochs=30,validation_data=(xTest,yTest), callbacks=model_checkpoint_callback)
model.fit(xTrain,yTrain,batch_size=8,epochs=20,validation_data=(xTest,yTest))


Epoch 1/20
90/90 [==============================] - 3s 27ms/step - loss: 0.7005 - accuracy: 0.5105 - val_loss: 0.6915 - val_accuracy: 0.5419
Epoch 2/20
90/90 [==============================] - 2s 22ms/step - loss: 0.6938 - accuracy: 0.5105 - val_loss: 0.6917 - val_accuracy: 0.5419
Epoch 3/20
90/90 [==============================] - 2s 22ms/step - loss: 0.6931 - accuracy: 0.5105 - val_loss: 0.6907 - val_accuracy: 0.5419
Epoch 4/20
90/90 [==============================] - 2s 22ms/step - loss: 0.6821 - accuracy: 0.6017 - val_loss: 0.6753 - val_accuracy: 0.5978
Epoch 5/20
90/90 [==============================] - 2s 22ms/step - loss: 0.6594 - accuracy: 0.6381 - val_loss: 0.6557 - val_accuracy: 0.6257
Epoch 6/20
90/90 [==============================] - 2s 22ms/step - loss: 0.6396 - accuracy: 0.6438 - val_loss: 0.7020 - val_accuracy: 0.6089
Epoch 7/20
90/90 [==============================] - 2s 24ms/step - loss: 0.6314 - accuracy: 0.6297 - val_loss: 0.6277 - val_accuracy: 0.6257
Epoch 8/20
90